# Check Python version and upgrading

In [78]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [95]:
import sys
import os

In [8]:
!conda update -n base -c defaults conda
!conda install -y pytorch torchvision -c pytorch

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/petrakov/anaconda3/envs/success

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1p             |       h5eee18b_0         2.5 MB
    pytorch-1.12.0             |py3.10_cuda11.3_cudnn8.3.2_0        1.19 GB  pytorch
    requests-2.28.0            |  py310h06a4308_0          94 KB
    torchvision-0.13.0         |      py310_cu113        28.3 MB  pytorch
    ------------------------------------------------------------
                                           Total:        1.22 GB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu
  blas             

In [ ]:
!pip uninstall -y numpy 
!pip install numpy

# Installations of 3 key libraries: KILT, GENRE and fairseq

In [5]:
#KILT
%rm -rf KILT
!git clone https://github.com/facebookresearch/KILT.git
%cd KILT
!pip install -r requirements.txt
!python setup.py install
sys.path.append(os.getcwd())
%cd ..

#GENRE
%rm -rf GENRE
!git clone https://github.com/facebookresearch/GENRE.git
%cd GENRE
!pip install ./
!python ./setup.py build develop install
!pip install sentencepiece marisa_trie
sys.path.append(os.getcwd())
%cd ..

# fairseq
%rm -rf fairseq
!git clone --branch fixing_prefix_allowed_tokens_fn https://github.com/nicola-decao/fairseq
%cd fairseq
!sed -i -e '26,27d' fairseq/registry.py
!pip install --editable ./
!python setup.py build develop
!python setup.py install
sys.path.append(os.getcwd())
%cd ..

!pip install kilt

#####
%cd fairseq
!pip install --editable ./
%cd ..

Cloning into 'KILT'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 401 (delta 108), reused 92 (delta 92), pack-reused 250
Receiving objects: 100% (401/401), 829.97 KiB | 3.32 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/home/petrakov/success/mGENRE_MEL/KILT
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Traceback (most recent call last):
  File "setup.py", line 8, in <module>
    import setuptools
ImportError: No module named setuptools
/home/petrakov/success/mGENRE_MEL
Cloning into 'GENRE'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 454 (delta 112), reused 104 (delta 80), pack-reused 284
Receiving objects: 100% (454/454), 10.99 MiB | 4.1

## import libraries

In [96]:
import gdown
import json
import pickle
import torch
from wikidata.client import Client

## Upload data and model (comment cell below if already downloaded)

In [7]:
# pretrained model
! rm -rf fairseq_multilingual_entity_disambiguation*
! wget https://dl.fbaipublicfiles.com/GENRE/fairseq_multilingual_entity_disambiguation.tar.gz
! tar -xvf fairseq_multilingual_entity_disambiguation.tar.gz

# data
! wget https://dl.fbaipublicfiles.com/GENRE/lang_title2wikidataID-normalized_with_redirect.pkl
! wget http://dl.fbaipublicfiles.com/GENRE/titles_lang_all105_marisa_trie_with_redirect.pkl

--2022-07-04 20:42:40--  https://dl.fbaipublicfiles.com/GENRE/fairseq_multilingual_entity_disambiguation.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2057741836 (1.9G) [application/gzip]
Saving to: ‘fairseq_multilingual_entity_disambiguation.tar.gz’

fairseq_multilingua 100%[===================>]   1.92G  20.8MB/s    in 96s     

2022-07-04 20:44:16 (20.5 MB/s) - ‘fairseq_multilingual_entity_disambiguation.tar.gz’ saved [2057741836/2057741836]

fairseq_multilingual_entity_disambiguation/
fairseq_multilingual_entity_disambiguation/model.pt
fairseq_multilingual_entity_disambiguation/dict.source.txt
fairseq_multilingual_entity_disambiguation/dict.target.txt
fairseq_multilingual_entity_disambiguation/spm_256000.model
fairseq_multilingual_entity_disambiguation/spm_256000.voca

## Install test data

In [8]:
id_ = "1QpLVXGahVjAoEq8TrE70aKtMLJCNeASh"
gdown.download(id=id_, output="mentions_test.json", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1QpLVXGahVjAoEq8TrE70aKtMLJCNeASh
To: /home/petrakov/success/mGENRE_MEL/mentions_test.json
100%|██████████| 65.1M/65.1M [00:00<00:00, 71.7MB/s]


'mentions_test.json'

In [9]:
with open("mentions_test.json") as f:
    test_set = json.load(f)

In original version there were serious of troubles that's why correct version we add here from external google drive

In [10]:
id__ = "1EfRYS0V3C0K7u_4SuP6DcEi31T_7dpq_"
gdown.download(id=id__, output="fairseq_model.py", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1EfRYS0V3C0K7u_4SuP6DcEi31T_7dpq_
To: /home/petrakov/success/mGENRE_MEL/fairseq_model.py
100%|██████████| 5.10k/5.10k [00:00<00:00, 2.32MB/s]


'fairseq_model.py'

In [11]:
%mv fairseq_model.py GENRE/genre

# Multilingual Autoregressive Entity Linking

![](s.png)

We are starting with importing our model mGENRE and it's important part, it was slightly changed in previous cells in order to reach stable work

In [1]:
from GENRE.genre.trie import Trie, MarisaTrie
from GENRE.genre.fairseq_model import mGENRE

Here we load data that connects titles with wikidata ids and trie that is an important part of the model

In [5]:
with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)
  
# with open("titles_lang_all105_trie_with_redirect.pkl", "rb") as f:
#     trie = Trie.load_from_dict(pickle.load(f))

with open("titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
    trie = pickle.load(f)

### Using GPU leads to faster inference, so if possible it is recommended

Here we chosen 4 because there were lot's of free memory on our own server, you can check the memory available for you using the command below

In [14]:
!nvidia-cdl

Driver version: 495.29.05
------------------- Device 0 -------------------
Name: NVIDIA GeForce RTX 3090
Memory usage: 22897MiB / 24268MiB
Temperature: 47C
Running processes not found

------------------- Device 1 -------------------
Name: NVIDIA GeForce RTX 3090
Memory usage: 18743MiB / 24268MiB
Temperature: 50C
Running processes not found

------------------- Device 2 -------------------
Name: NVIDIA GeForce RTX 3090
Memory usage: 16130MiB / 24268MiB
Temperature: 45C
Running processes not found

------------------- Device 3 -------------------
Name: NVIDIA GeForce RTX 3090
Memory usage: 18743MiB / 24268MiB
Temperature: 52C
Running processes not found

------------------- Device 4 -------------------
Name: NVIDIA GeForce RTX 2080 Ti
Memory usage: 10549MiB / 11019MiB
Temperature: 27C
Running processes not found

------------------- Device 5 -------------------
Name: NVIDIA GeForce RTX 2080 Ti
Memory usage:  8127MiB / 11019MiB
Temperature: 74C
Running processes not found



In [18]:
device = "cpu"#torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [19]:
# generate Wikipedia titles and language IDs
model = mGENRE.from_pretrained("fairseq_multilingual_entity_disambiguation").eval()
model.to(device)

GENREHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(256001, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDro

Example of model result

In [20]:
sentences = ["[START] The founder of the theory of relativity [END] received the Nobel Prize."]
model.sample(
    sentences,
    beam = 5,
    prefix_allowed_tokens_fn=lambda batch_id, sent: [
        e for e in trie.get(sent.tolist())
        if e < len(model.task.target_dictionary)
    ],
    text_to_id=lambda x: max(lang_title2wikidataID[tuple(reversed(x.split(" >> ")))], key=lambda y: int(y[1:])),
    marginalize=True,
    verbose = True
)

/home/petrakov/success/mGENRE_MEL/fairseq/fairseq/search.py:205: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/home/petrakov/success/mGENRE_MEL/fairseq/fairseq/sequence_generator.py:659: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


[[{'id': 'Q937',
   'texts': ['Albert Einstein >> en'],
   'scores': tensor([-0.7139]),
   'score': tensor(-1.5963)},
  {'id': 'Q363005',
   'texts': ['John Singleton >> en'],
   'scores': tensor([-0.7911]),
   'score': tensor(-1.9379)},
  {'id': 'Q7374',
   'texts': ['Alfred Hitchcock >> en'],
   'scores': tensor([-0.8763]),
   'score': tensor(-2.3185)},
  {'id': 'Q715110',
   'texts': ['Robert Maynard >> en'],
   'scores': tensor([-0.8955]),
   'score': tensor(-2.3693)},
  {'id': 'Q9353',
   'texts': ['John Locke >> en'],
   'scores': tensor([-1.0061]),
   'score': tensor(-2.4643)}]]

Results above demonstrate that each *text* has it's own id and via marginalization procedure we have unique score for predicted entity

# New experiment GENRE

In [23]:
# ## Data and dependencies

# id_p_e_m = "1C2R814tsgZbREaaOk6o9nh3lQn308Wo7"
# gdown.download(id=id_p_e_m, output="prob_yago_crosswikis_wikipedia_p_e_m.txt", quiet=False)

# %mkdir data
# %cd data
# %mkdir dalab
# %cd ..

# %mv prob_yago_crosswikis_wikipedia_p_e_m.txt data/dalab/prob_yago_crosswikis_wikipedia_p_e_m.txt

# ! wget http://resources.mpi-inf.mpg.de/yago-naga/aida/download/aida_means.tsv.bz2
# ! bzip2 -dk aida_means.tsv.bz2

# %cd data 
# %mkdir aida
# %cd ..

# %mv aida_means.tsv data/aida/aida_means.tsv

In [24]:
# def read_dalab_candidates():
#     for line in open("data/dalab/prob_yago_crosswikis_wikipedia_p_e_m.txt"):
#         line = line[:-1]
#         columns = line.split("\t")
#         mention = columns[0]
#         for column in columns[2:]:
#             if len(column.strip()) == 0:
#                 continue
#             values = column.split(",")
#             candidate = ",".join(values[2:])
#             candidate = candidate.replace("_", " ")
#             yield mention, candidate


# def hex2int(hexa: str) -> int:
#     return int(hexa, 16)


# def replace_unicode(u_str):
#     matches = set(re.findall("\\\\u....", u_str))
#     for match in matches:
#         u_str = u_str.replace(match, chr(hex2int(match[2:])))
#     return u_str


# PUNCTUATION_CHARS = set(string.punctuation)


# def filter_mention(mention):
#     if mention[0].islower():
#         return True
#     if mention[0] in PUNCTUATION_CHARS:
#         return True
#     return False


# def read_aida_candidates():
#     for line in open("data/aida/aida_means.tsv"):
#         line = line[:-1]
#         values = line.split("\t")
#         mention = replace_unicode(values[0][1:-1])
#         candidate = replace_unicode(values[1]).replace("_", " ")
#         yield mention, candidate


# #making mention_candidates_dict
# #once done no need to do again

# mention_candidates_dict = {}
# for mention, candidate in itertools.chain(read_dalab_candidates(), read_aida_candidates()):
#     if filter_mention(mention):
#         continue
#     if mention not in mention_candidates_dict:
#         mention_candidates_dict[mention] = set()
#     mention_candidates_dict[mention].add(candidate)
# for mention in mention_candidates_dict:
#     mention_candidates_dict[mention] = sorted(mention_candidates_dict[mention])
# with open("data/mention_candidates_dict.pkl", "wb") as f:
#     pickle.dump(mention_candidates_dict, f)

In [25]:
# %mkdir models

In [26]:
# # making mention_trie
# # once done no need to do again

# sys.setrecursionlimit(10000)
# model_path = "fairseq_e2e_entity_linking_wiki_abs"
# model = GENRE.from_pretrained(model_path).eval()
# with open("data/mention_candidates_dict.pkl", "rb") as f:
#     mention_to_candidates_dict = pickle.load(f)
# mention_trie = Trie()
# for mention in tqdm(mention_to_candidates_dict):
#     encoded = model.encode(" {}".format(mention))[1:].tolist()
#     mention_trie.add(encoded)
# out_file = "data/mention_trie.pkl"
# with open(out_file, "wb") as f:
#     pickle.dump(mention_trie, f)

### example from git

In [27]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import Markdown
import pickle
import sys

from GENRE.genre.utils import get_entity_spans_fairseq as get_entity_spans
from GENRE.genre.utils import get_markdown


from GENRE.genre.trie import Trie, MarisaTrie
from GENRE.genre.fairseq_model import mGENRE
from GENRE.genre.fairseq_model import GENRE


model_path_genre = "fairseq_e2e_entity_linking_wiki_abs"
model_path_mgenre = "fairseq_multilingual_entity_disambiguation"
dict_path = "data/mention_candidates_dict.pkl"
trie_path = "data/mention_trie.pkl"

model = GENRE.from_pretrained(model_path_genre).eval()

with open(trie_path, "rb") as f:
    mention_trie = pickle.load(f)
with open(dict_path, "rb") as f:
    mention_to_candidates_dict = pickle.load(f)

text = """Home Depot CEO Nardelli quits Home-improvement retailer's chief executive had been criticized over pay ATLANTA - Bob Nardelli abruptly resigned Wednesday as chairman and chief executive of The Home Depot Inc. after a six-year tenure that saw the world’s largest home improvement store chain post big profits but left investors disheartened by poor stock performance. Nardelli has also been under fire by investors for his hefty pay and is leaving with a severance package valued at about $210 million. He became CEO in December 2000 after being passed over for the top job at General Electric Co., where Nardelli had been a senior executive. Home Depot said Nardelli was being replaced by Frank Blake, its vice chairman, effective immediately. Blake’s appointment is permanent, Home Depot spokesman Jerry Shields said. What he will be paid was not immediately disclosed, Shields said. The company declined to make Blake available for comment, and a message left for Nardelli with his secretary was not immediately returned. Before Wednesday’s news, Home Depot’s stock had been down more than 3 percent on a split-adjusted basis since Nardelli took over. Nardelli’s sudden departure was stunning in that he told The Associated Press as recently as Sept. 1 that he had no intention of leaving, and a key director also said that the board was pleased with Nardelli despite the uproar by some investors. Asked in that interview if he had thought of hanging up his orange apron and leaving Home Depot, Nardelli said unequivocally that he hadn’t. Asked what he thought he would be doing 10 years from now, Nardelli said, “Selling hammers.” For The Home Depot? “Absolutely,” he said at the time. Home Depot said Nardelli’s decision to resign was by mutual agreement with the Atlanta-based company. “We are very grateful to Bob for his strong leadership of The Home Depot over the past six years. Under Bob’s tenure, the company made significant and necessary investments that greatly improved the company’s infrastructure and operations, expanded our markets to include wholesale distribution and new geographies, and undertook key strategic initiatives to strengthen the company’s foundation for the future,” Home Depot’s board said in a statement. Nardelli was a nuts-and-bolts leader, a former college football player and friend of President Bush. He helped increase revenue and profits at Home Depot and increase the number of stores the company operates to more than 2,000. Home Depot’s earnings per share have increased by approximately 150 percent over the last five years."""

sentences = [text]
entity_spans = get_entity_spans(
    model,
    sentences,
    mention_trie=mention_trie,
    mention_to_candidates_dict=mention_to_candidates_dict
)
markdown = get_markdown(sentences, entity_spans)[0]

In [28]:
Markdown(markdown)

[Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) CEO Nardelli quits Home-improvement retailer's chief executive had been criticized over pay ATLANTA - [Bob Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) abruptly resigned Wednesday as chairman and chief executive of [The Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) Inc. after a six-year tenure that saw the world’s largest home improvement store chain post big profits but left investors disheartened by poor stock performance. [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) has also been under fire by investors for his hefty pay and is leaving with a severance package valued at about $210 million. He became CEO in December 2000 after being passed over for the top job at [General Electric](https://en.wikipedia.org/wiki/General_Electric) Co., where [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) had been a senior executive. [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) said [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) was being replaced by [Frank Blake](https://en.wikipedia.org/wiki/Frank_Blake), its vice chairman, effective immediately. [Blake](https://en.wikipedia.org/wiki/Frank_Blake)’s appointment is permanent, [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) spokesman [Jerry Shields](https://en.wikipedia.org/wiki/Jerry_A._Shields) said. [What](https://en.wikipedia.org/wiki/What?_(film)) he will be paid was not immediately disclosed, [Shields](https://en.wikipedia.org/wiki/Jerry_A._Shields) said. [The](https://en.wikipedia.org/wiki/The_Home_Depot) company declined to make [Blake](https://en.wikipedia.org/wiki/Frank_Blake) available for comment, and a message left for [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) with his secretary was not immediately returned. [Before](https://en.wikipedia.org/wiki/Before_(song)) Wednesday’s news, [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot)’s stock had been down more than 3 percent on a split-adjusted basis since [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) took over. [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli)’s sudden departure was stunning in that he told [The](https://en.wikipedia.org/wiki/The_Home_Depot) Associated Press as recently as Sept. 1 that he had no intention of leaving, and a key director also said that the board was pleased with [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) despite the uproar by some investors. Asked in that interview if he had thought of hanging up his orange apron and leaving [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot), [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) said unequivocally that he hadn’t. Asked what he thought he would be doing [10](https://en.wikipedia.org/wiki/10_(film)) years from now, [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli) said, “Selling hammers.” For [The](https://en.wikipedia.org/wiki/The_Mr._T_Experience) Home Depot? “Absolutely,” he said at the time. [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) said [Nardelli](https://en.wikipedia.org/wiki/Robert_Nardelli)’s decision to resign was by mutual agreement with the [Atlanta](https://en.wikipedia.org/wiki/Decatur,_Georgia)-based company. “We are very grateful to [Bob](https://en.wikipedia.org/wiki/Barbecue_Bob) for his strong leadership of [The](https://en.wikipedia.org/wiki/U.S._Department_of_State_Foreign_Affairs_Manual) [Home Depot](https://en.wikipedia.org/wiki/The_Home_Depot) over the past six years. Under [Bob](https://en.wikipedia.org/wiki/Bank_of_Baroda)’s tenure, the company made significant and necessary investments that greatly improved the company’s infrastructure and operations, expanded our markets to include wholesale distribution and new geographies, and undertook key strategic initiatives to strengthen the company’s foundation for the future,” [Home](https://en.wikipedia.org/wiki/John_Home) [Depot](https://en.wikipedia.org/wiki/Depot)’s board said in a statement. [Nardelli](https://en.wikipedia.org/wiki/Nardelli) was a nuts-and-bolts leader, a former college football player and friend of [President](https://en.wikipedia.org/wiki/Senegalese_presidential_election,_2007) Bush. [He](https://en.wikipedia.org/wiki/J._M._E._McTaggart) helped increase revenue and profits at [Home](https://en.wikipedia.org/wiki/New_Recreation_Ground) [Depot](https://en.wikipedia.org/wiki/Depot) and increase the number of stores the company operates to more than [2](https://en.wikipedia.org/wiki/U.S._Route_2),000. [Home](https://en.wikipedia.org/wiki/New_Recreation_Ground) [Depot](https://en.wikipedia.org/wiki/Depot)’s earnings per share have increased by approximately [150](https://en.wikipedia.org/wiki/U.S._Route_150) percent over the last five years.

# Estimate scores

In [29]:
documents = {
    "id_0": "In 1921, Einstein received a Nobel Prize.",
    "id_1": "Armstrong was the first man on the Moon.",
}

gold_entities = [
    ("id_0", 3, 4, "1921"),
    ("id_0", 9, 8, 'Albert_Einstein'),
    ("id_0", 29, 11, 'Nobel_Prize_in_Physics'),
    ("id_1", 0, 9, 'Neil_Armstrong'),
    ("id_1", 35, 4, 'Moon'),
]

In [33]:
guess_entities = get_entity_spans(
    model,
    list(documents.values()),
)
print("guess_entities", guess_entities)
print("#############")

guess_entities = [
    (k,) + x
    for k, e in zip(documents.keys(), guess_entities)
    for x in e
]
print("guess_entities", guess_entities)

guess_entities [[(0, 7, 'List_of_Nobel_laureates_by_year_of_appointment'), (9, 8, 'Albert_Einstein'), (29, 11, 'Nobel_Prize_in_Physiology_or_Medicine'), (40, 1, 'List_of_Nobel_laureates_in_Physiology_or_Medicine_by_year_of_appointment')], [(18, 9, 'First_Man_(film)'), (35, 4, 'Moon_(TV_series)_(1968_TV_series,_season_1)'), (39, 1, 'Moon_(TV_series,_season_1)')]]
#############
guess_entities [('id_0', 0, 7, 'List_of_Nobel_laureates_by_year_of_appointment'), ('id_0', 9, 8, 'Albert_Einstein'), ('id_0', 29, 11, 'Nobel_Prize_in_Physiology_or_Medicine'), ('id_0', 40, 1, 'List_of_Nobel_laureates_in_Physiology_or_Medicine_by_year_of_appointment'), ('id_1', 18, 9, 'First_Man_(film)'), ('id_1', 35, 4, 'Moon_(TV_series)_(1968_TV_series,_season_1)'), ('id_1', 39, 1, 'Moon_(TV_series,_season_1)')]


In [31]:
from GENRE.genre.utils import (
    get_micro_precision,
    get_micro_recall,
    get_micro_f1,
    get_macro_precision,
    get_macro_recall,
    get_macro_f1,
)

micro_p = get_micro_precision(guess_entities, gold_entities)
micro_r = get_micro_recall(guess_entities, gold_entities)
micro_f1 = get_micro_f1(guess_entities, gold_entities)
macro_p = get_macro_precision(guess_entities, gold_entities)
macro_r = get_macro_recall(guess_entities, gold_entities)
macro_f1 = get_macro_f1(guess_entities, gold_entities)

print(
   "micro_p={:.4f} micro_r={:.4f}, micro_f1={:.4f}, macro_p={:.4f}, macro_r={:.4f}, macro_f1={:.4f}".format(
       micro_p, micro_r, micro_f1, macro_p, macro_r, macro_f1
   )
)

micro_p=0.1429 micro_r=0.2000, micro_f1=0.1667, macro_p=0.1250, macro_r=0.1667, macro_f1=0.1429


# load mewsli-9

In [4]:
import pandas as pd

In [5]:
%ls

aida_means.tsv.bz2
data/
dataset/
docs.tsv
fairseq/
fairseq_e2e_entity_linking_wiki_abs/
fairseq_multilingual_entity_disambiguation/
fairseq_multilingual_entity_disambiguation.tar.gz
GENRE/
KILT/
lang_title2wikidataID-normalized_with_redirect.pkl
mentions_test.json
mentions.tsv
mewsli_9/
mgenre_final.ipynb
mgenre_final.ipynb.invalid
README.md
requirements.txt
titles_lang_all105_marisa_trie_with_redirect.pkl


In [8]:
#en_df = pd.read_csv('dataset/en/mentions.tsv', sep='\t')
#en_df_doc = pd.read_csv('docs.tsv', sep='\t')
en_df_doc = pd.read_csv('mewsli_9/dense_representations_for_entity_retrieval/mel/mewsli-9/output/dataset/en/docs.tsv', sep='\t')
en_df_men = pd.read_csv('mewsli_9/dense_representations_for_entity_retrieval/mel/mewsli-9/output/dataset/en/mentions.tsv', sep='\t')



In [7]:
en_df_doc

,docid,title,curid,revid,url,text_md5
0,en-106602,NOAA says Earth's oceans becoming more acidic,106602,1986523,https://en.wikinews.org/wiki?curid=106602,a504b8679bd5f6eeae93536d2b3bb6d6
1,en-106608,Two men fined over 2006 German train crash tha...,106608,1100545,https://en.wikinews.org/wiki?curid=106608,b183ca9d8392849395cd56e37c2a5cb4
2,en-106610,New law to help asbestos sufferers in Victoria...,106610,4360069,https://en.wikinews.org/wiki?curid=106610,86b30cb07eea2779541562c54eb9261e
3,en-106632,President Bush to meet with LDS Church leaders...,106632,1410456,https://en.wikinews.org/wiki?curid=106632,b5eefea779c04081602521049e579b9f
4,en-106644,Morgan Tsvangirai returns to Zimbabwe,106644,4366347,https://en.wikinews.org/wiki?curid=106644,7aead039ec500822f9a7c4eeae120963
...,...,...,...,...,...,...
12674,en-82692,Fußball-Bundesliga 2007–08: Borussia Dortmund ...,82692,4402330,https://en.wikinews.org/wiki?curid=82692,cf7b4f3abfca4b2d26fab8e86aec2e20
12675,en-82698,"Blast kills 8, injures 126 at a mall in Manila...",82698,1628197,https://en.wikinews.org/wiki?curid=82698,019d12477b221f7b78bb839f83f58733
12676,en-82700,"USPTO partially confirms validity of Amazon ""1...",82700,2470912,https://en.wikinews.org/wiki?curid=82700,fbce4beee7f28115a0811fe8fead83f2
12677,en-82707,Ben Cousins sacked by West Coast in AFL drug s...,82707,2515447,https://en.wikinews.org/wiki?curid=82707,8e2efe3f69fb5981a89b8e30bac51d40


In [9]:
en_df_men

,docid,position,length,mention,url,lang,qid,qid_in_refs,freq_bin
0,en-106602,145,39,Pacific Marine Environmental Laboratory,http://en.wikipedia.org/wiki/Pacific_Marine_En...,en,Q7122548,True,2
1,en-106602,365,15,Baja California,http://en.wikipedia.org/wiki/Baja_California,en,Q58731,True,5
2,en-106602,1013,13,oceanographer,http://en.wikipedia.org/wiki/Oceanography,en,Q43518,True,4
3,en-106602,1109,5,Earth,http://en.wikipedia.org/wiki/Earth,en,Q2,True,5
4,en-106602,1150,14,carbon dioxide,http://en.wikipedia.org/wiki/Carbon_dioxide,en,Q1997,True,5
...,...,...,...,...,...,...,...,...,...
80237,en-82707,99,3,AFL,http://en.wikipedia.org/wiki/Australian_Footba...,en,Q50783,True,5
80238,en-82707,501,6,Valium,http://en.wikipedia.org/wiki/Valium,en,Q210402,True,4
80239,en-82713,161,6,reggae,http://en.wikipedia.org/wiki/Reggae,en,Q9794,True,5
80240,en-82713,173,10,Lucky Dube,http://en.wikipedia.org/wiki/Lucky_Dube,en,Q380147,True,3


In [33]:
%cd mewsli_9/dense_representations_for_entity_retrieval/mel

/home/petrakov/success/mGENRE_MEL/mewsli_9/dense_representations_for_entity_retrieval/mel


In [34]:
!pip install -r wikinews_extractor/requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [39]:
%cd ..

/home/petrakov/success/mGENRE_MEL/mewsli_9/dense_representations_for_entity_retrieval/mel


In [40]:
!bash mewsli-9/run_parse_wikinews_i18n.sh

++ dirname mewsli-9/run_parse_wikinews_i18n.sh
+ INPUT_BASE_DIR_DEFAULT=mewsli-9/output/wikiextractor
+ INPUT_BASE_DIR=mewsli-9/output/wikiextractor
++ dirname mewsli-9/run_parse_wikinews_i18n.sh
+ OUTPUT_BASE_DIR_DEFAULT=mewsli-9/output/dataset
+ OUTPUT_BASE_DIR=mewsli-9/output/dataset
+ LANG_LIST=(ar de en es fa ja sr ta tr)
+ MODULE=dense_representations_for_entity_retrieval.mel.wikinews_extractor.parse_wikinews_i18n
+ for lang in ${LANG_LIST[@]}
+ date
Tue 05 Jul 2022 12:36:54 AM MSK
+ echo '>Parse '\''ar'\''...'
>Parse 'ar'...
+ output_dir=mewsli-9/output/dataset/ar
+ mkdir -p mewsli-9/output/dataset/ar
+ echo

+ for lang in ${LANG_LIST[@]}
+ date
+ python -m dense_representations_for_entity_retrieval.mel.wikinews_extractor.parse_wikinews_i18n --mode=dataset '--wikinews_archive=mewsli-9/output/wikiextractor/ar/*/wiki_*.bz2' --language=ar --mention_index_path=mewsli-9/output/dataset/ar/mentions.tsv --doc_index_path=mewsli-9/output/dataset/ar/docs.tsv --output_dir_wiki=mewsli-9/outp

In [23]:
%ls

get-mewsli-9.sh  mewsli-9/    mewsli-9.zip  mewsli-x.md  tools/
get-mewsli-x.sh  mewsli-9.md  mewsli_x/     README.md    wikinews_extractor/


In [ ]:
./mewsli-9/output/dataset/es/text/es-44

In [44]:
import gdown

In [80]:
import bz2

In [97]:
with bz2.open("mewsli-9/output/download/enwikinews-20190101-pages-articles.xml.bz2", "rb") as f:
    # Decompress data from file
    en_wiki = f.read()



In [98]:
import pandas as pd

In [99]:
pd.DataFrame(en_wiki.decode())

ValueError: DataFrame constructor not properly called!

In [86]:
pd.read_xml("mewsli-9/output/download/enwikinews-20190101-pages-articles.xml.bz2")

XMLSyntaxError: internal error: Huge input lookup, line 3616783, column 11 (<string>, line 3616783)

### Another example

In [24]:
client = Client()  # doctest: +SKIP
entity = client.get('Q20145', load=True)
entity, entity.description

(<wikidata.entity.Entity Q20145 'IU'>,
 m'Korea  singer,actress record producer (2017)')

In [25]:
class bcolors:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Below we present a function that illustrates predicted entity, text, right answer basing on the id as input

In [36]:
def predict_from_test_set(idx=0):
    print(test_set[str(idx)]['mention_its']+"\n")
    start = test_set[str(idx)]['start_index']
    end = test_set[str(idx)]['end_index']
    mention_id = test_set[str(idx)]['mention_id']
    text = test_set[str(idx)]['source_document']['text']
    text = text[:start]+"[START] "+text[start:end]+" [END]"+text[end:]
    print(f'{text[:start]}{bcolors.BOLD}{text[start:end+len("[START] ")+len(" [END]")]}{bcolors.END}{text[end+len("[START] ")+len(" [END]"):]}\n')
    result = model.sample(
    sentences=[text],
    text_to_id=lambda x: sorted(list(lang_title2wikidataID.get(
        tuple(reversed((x.split(" >> ")[0], x.split(" >> ")[1][:2]))), [None])))[0],
    marginalize=True)
    print(result)
    
    entity = client.get(mention_id, load=True)
    print(f'\nCorrect entity : {mention_id, entity.label, entity.description}\n')

    candidates = [(i['id'], client.get(i['id'], load=True)) for i in result[0]]
    print('Predicted entities:')
    for i, entity in candidates:
        if i is not None:
            print(f'{i, entity.label, entity.description}')

In [37]:
predict_from_test_set(2354)

New South Wales

Bathurst Regional Council, the local government responsible for the city of Bathurst and its surrounds in Central Western [START] New South Wales [END], Australia yesterday revealed it had received a development application for the new Bathurst Base Hospital.The new hospital is to be built behind the current hospital on the same site and is expected to cost the New South Wales government AUD96 million. The Bathurst Hospital will be the first in the Bathurst-Orange-Bloomfield redevelopment project.The new hospital will have 149 beds, up from 85 for the current hospital. The hospital will also feature a mental health unit - previously psychiatric patients had to travel to Orange to the Bloomfield Hospital for treatment.The Bathurst Hospital is expected to have state-of-the art facilities and will share some services with the to be constructed Orange Base Hospital.The Bathurst Regional Council has approved the demolition of 12 buildings on the hospital site for enabling w

# Preliminary results:

Language: **fa** (5 out of 61)

Micro average (k=1): **1.0**

Language: **sr** (105 out of 451)

Micro average (k=1): **0.90476**

Language: **ta** (25 out of 366)

Micro average (k=1): **1.0**

### Macro average (k=1): 0.96825

In [28]:
from tqdm import tqdm
from multiprocessing import Pool

In [29]:
from fairseq.data.data_utils import collate_tokens


def compute_results(idx):
    start = test_set[str(idx)]['start_index']
    end = test_set[str(idx)]['end_index']
    mention_id = test_set[str(idx)]['mention_id']
    text = test_set[str(idx)]['source_document']['text']
    text = text[:start]+"[START] "+text[start:end]+" [END]"+text[end:]
    print(start, end, min(start, len(text)-end))
    #encoder = model.encoder(text)
    #print(encoder)

    result = model.sample(
                    sentences=[text],
                   text_to_id=lambda x: sorted(list(lang_title2wikidataID.get(
       tuple(reversed((x.split(" >> ")[0], x.split(" >> ")[1][:2]))), [None])))[0],
                    marginalize=True)
    candidates = [(i['id'], client.get(i['id'], load=True)) for i in result[0] if i['id'] is not None]
    entity = client.get(mention_id, load=True)
    return {
            "correct": mention_id, 
            "predicted": [i for i, _ in candidates]
               }

In [30]:
compute_results(1)

100 109 100


{'correct': 'Q36678', 'predicted': ['Q36678', 'Q2564150', 'Q7834492']}

In [32]:
def compute_scores(lang='en', k_list=[1, 10, 20, 50, 100]):
    scores = []
    
    for idx in tqdm(langs_idx[lang]):
        scores.append(compute_results(idx))
    return scores

In [33]:
langs_idx = {
    'ar':[], 'en':[], 'tr':[]
}
for idx in test_set:
    el = test_set[idx]
    lang = el['document_id'][:2]
    if lang in langs_idx:
        langs_idx[lang].append(int(idx))

In [34]:
all_langs_micro_avg = []
print('Model F:')
print('on test dataset:\n')

lang2scores = {}

for l in langs_idx:
    lang2scores[l] = compute_scores(l)

Model F:
on test dataset:



  0%|          | 0/1799 [00:00<?, ?it/s]

738 744 14


  0%|          | 1/1799 [00:01<51:19,  1.71s/it]

350 357 350


...

  0%|          | 7/1799 [00:14<1:08:25,  2.29s/it]

166 172 166


  0%|          | 8/1799 [00:16<1:00:18,  2.02s/it]

1035 1041 1035


Exception: Size of sample #0 is invalid (=(2949, 0)) since max_positions=(1024, 1024), skip this example with --skip-invalid-size-inputs-valid-test

In [ ]:
def micro_avg_lang(scores, lang='en', k_list=[1, 10, 20, 50, 100]):    
    print(f'Language: {lang}')
    results = []
    for k in k_list:
        n = 0
        for i, row in enumerate(scores):
            if scores['correct'] in scores['predicted'][:k]:
                n += 1
        results.append(round(n / total_mentions, 5))
    for i, k in enumerate(k_list):
        print(f'Micro average (k={k}):', results[i])
    return results